In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from PIL import Image
from scipy import sparse
import cv2
from pymatreader import read_mat
# from extract_graph import dic_to_sparse
from util import get_path
from plotutil import show_im,overlap, show_im_rgb, plot_nodes, plot_nodes_from_list,plot_t_tp1
from extract_graph import generate_graph_tab_from_skeleton,generate_nx_graph_from_skeleton,generate_skeleton,clean
import networkx as nx
from node_id import second_identification, whole_movement_identification,first_identification,relabel_nodes, clean_nodes, orient
plate=13
date1='0703_1157'
date2='0703_1557'
date3='0703_1957'
row=6
column=10


In [2]:
imtab1=np.load(f'Data/imbackrem_{date1}_{plate}_{row}_{column}.npy')
imtab2=np.load(f'Data/imbackrem_{date2}_{plate}_{row}_{column}.npy')
imtab3=np.load(f'Data/imbackrem_{date3}_{plate}_{row}_{column}.npy')
skeleton1=clean(np.load(f'Data/skeletonized_{date1}_{plate}_{row}_{column}.npy'))
skeleton2=clean(np.load(f'Data/skeletonized_{date2}_{plate}_{row}_{column}.npy'))
skeleton3=clean(np.load(f'Data/skeletonized_{date3}_{plate}_{row}_{column}.npy'))
width2=np.load(f'Data/width_{date2}_{plate}_{row}_{column}.npy')

In [3]:
agraph_tab1=generate_graph_tab_from_skeleton(skeleton1)
agraph_tab2=generate_graph_tab_from_skeleton(skeleton2)
agraph_tab3=generate_graph_tab_from_skeleton(skeleton3)
anx_graph_t0m1,apos_t0m1=generate_nx_graph_from_skeleton(skeleton1)
anx_graph_t0,apos_t0=generate_nx_graph_from_skeleton(skeleton2)
anx_graph_t0p1,apos_t0p1=generate_nx_graph_from_skeleton(skeleton3)

In [4]:

# pos_t0p1_clean,nx_graph_t0p1_clean,nx_graph_t0_clean,corresp_tips=second_identification(anx_graph_t0,anx_graph_t0p1,apos_t0,apos_t0p1,50)
# pos_t0_clean,nx_graph_t0_clean_clean,nx_graph_t0m1_clean,corresp_tips=second_identification(anx_graph_t0m1,nx_graph_t0_clean,apos_t0m1,apos_t0,50)
# skeleton_clean_t0m1=generate_skeleton(nx_graph_t0m1_clean).todense()
# skeleton_clean_t0=generate_skeleton(nx_graph_t0_clean_clean).todense()
# skeleton_clean_t0p1=generate_skeleton(nx_graph_t0p1_clean).todense()

In [5]:
apos_t0,anx_graph_t0,anx_graph_t0m1,corresp_tips=second_identification(anx_graph_t0m1,anx_graph_t0,apos_t0m1,apos_t0,50)
apos_t0p1,anx_graph_t0p1,anx_graph_t0,corresp_tips=second_identification(anx_graph_t0,anx_graph_t0p1,apos_t0,apos_t0p1,50)



skeleton_clean_t0m1=generate_skeleton(anx_graph_t0m1).todense()
skeleton_clean_t0=generate_skeleton(anx_graph_t0).todense()
skeleton_clean_t0p1=generate_skeleton(anx_graph_t0p1).todense()

{49}
{33, 322, 35, 337, 24, 313, 314, 315, 316, 317, 25}


In [21]:
tip_corresp=whole_movement_identification(anx_graph_t0m1,anx_graph_t0,apos_t0m1,apos_t0,50)

In [7]:
# list_anastomosis = pinpoint_anastomosis(anx_graph_t0m1,anx_graph_t0)
list_tips = [node for node in anx_graph_t0m1.nodes if anx_graph_t0m1.degree(node)==1]
list_tips2 = [node for node in anx_graph_t0.nodes if anx_graph_t0.degree(node)==1]

In [16]:
pixel_conversion_factor = 1.725
kernel = np.ones((5,5),np.uint8)
dilated2 = cv2.dilate(width2.astype(np.uint8) *pixel_conversion_factor,kernel,iterations = 2)

In [42]:
list_nodes_t

In [8]:
def relabel(tip):
    return((list_tips+list_tips2).index(tip))

In [17]:
test=(dilated2>60)*60+((dilated2<=60)*dilated2*(dilated2>=20))+(dilated2<20)*20

In [26]:
plot_t_tp1(list_tips,list_tips,apos_t0m1,apos_t0,imtab2,imtab2,relabel_t=relabel,relabel_tp1=lambda x : relabel(tip_corresp[x]) if x in tip_corresp.keys() else relabel(x))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
plot_t_tp1([],[],apos_t0m1,apos_t0,dilated2,imtab2,relabel_t=relabel,relabel_tp1=lambda x : relabel(tip_corresp[x]) if x in tip_corresp.keys() else relabel(x))

NameError: name 'dilated2' is not defined

In [73]:
plot_t_tp1(list_tips,list_tips+list_tips2,apos_t0m1,apos_t0,dilated2,imtab2,relabel_t=relabel,relabel_tp1=lambda x : relabel(tip_corresp[x]) if x in tip_corresp.keys() else relabel(x))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
list_tipstm1=[tip for tip in list_tips if (apos_t0m1[tip][0]<=2500 and apos_t0m1[tip][0]>=500 and apos_t0m1[tip][1]<=1500)]

In [20]:
list_tipst=[tip for tip in list_tips2 if (apos_t0[tip][0]<=2500 and apos_t0[tip][0]>=500 and apos_t0[tip][1]<=1500)]

In [100]:
def plot_t_tp1(node_list_t,node_list_tp1,pos_t,pos_tp1,imt,imtp1,relabel_t=lambda x:x,relabel_tp1=lambda x:x):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.imshow(imtp1, cmap='gray',interpolation='none',extent=[0,imtp1.shape[1]*pixel_conversion_factor,imtp1.shape[0]*pixel_conversion_factor,0])
    width=ax.imshow(imt, cmap='jet', alpha=0.5,interpolation='none',extent=[0,imtp1.shape[1]*pixel_conversion_factor,imtp1.shape[0]*pixel_conversion_factor,0])
    bbox_props1 = dict(boxstyle="circle", fc="grey")
    bbox_props2 = dict(boxstyle="circle", fc="white")
    for node in node_list_t:
        t = ax.text(pos_t[node][1]*pixel_conversion_factor, pos_t[node][0]*pixel_conversion_factor, str(relabel_t(node)), ha="center", va="center",
                    size=5,
                    bbox=bbox_props1)
    for node in node_list_tp1:
        if node in pos_tp1.keys():
            t = ax.text(pos_tp1[node][1]*pixel_conversion_factor, pos_tp1[node][0]*pixel_conversion_factor, 
                        str(relabel_tp1(node)), ha="center", va="center",
                        size=5,
                        bbox=bbox_props2)
    ax.set_xlabel('x position (μm)')
    ax.set_ylabel('y position (μm)')
    cbar = fig.colorbar(width,ax=ax)
    cbar.set_label('Hyphal width (μm)')
    plt.show()

In [6]:
def pinpoint_anastomosis(nx_graph_tm1,nx_grapht):
    anastomosis=[]
    tips = [node for node in nx_graph_tm1.nodes if nx_graph_tm1.degree(node)==1]
    for node in tips:
        if nx_grapht.degree(node)>=3:
            anastomosis.append(node)
    return(anastomosis)

In [59]:
kernel = np.ones((5,5),np.uint8)
pixel_conversion_factor = 1.725
dilated2 = cv2.dilate(width2.astype(np.uint8) *pixel_conversion_factor,kernel,iterations = 1)

In [ ]:
plot_t_tp1(list_tips,list_tips,apos_t0m1,apos_t0,dilated2,imtab2)

In [10]:
plot_t_tp1(anx_graph_t0m1.nodes,anx_graph_t0.nodes,apos_t0m1,apos_t0,skeleton1,imtab1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
distance = [np.linalg.norm(np.array(apos_t0m1[tip])-np.array(apos_t0[tip]))*1.725/(4*3600) for tip in list_tips]

In [12]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.hist(distance)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
plot_t_tp1(list_tips2,list_tips2,apos_t0,apos_t0p1,imtab2,imtab3)

In [3]:
date1='0709_1557'
date2='0705_1557'
date3='0705_1957'


In [4]:
get_path(date1,plate,True)

'//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE/20200709_1557_Plate13/Analysis/Skeleton.mat'

In [5]:
mat_skel=read_mat(get_path(date1,plate,True))['skel']

In [6]:
from extract_graph import dic_to_sparse, from_sparse_to_graph, generate_nx_graph, prune_graph


In [7]:
dic_skel=dic_to_sparse(mat_skel)

In [10]:
skeleton_doc1=sparse.dok_matrix(dic_skel)

In [9]:
skeleton_doc.shape

NameError: name 'skeleton_doc' is not defined

In [73]:
skeleton_doc_small[500]

IndexError: row index (30) out of range

In [59]:
skeleton_doc_small=skeleton_doc1[15000:20000,15000:20000]

# graph_tab_small=from_sparse_to_graph(skeleton_doc_small)
# nx_graph_small,pos_small=generate_nx_graph(graph_tab_small)

In [60]:
skeleton_doc_1_dilated=dilate(skeleton_doc_small)

In [40]:
from extract_graph import from_sparse_to_graph, generate_nx_graph
graph_tab_1_reconnected=from_sparse_to_graph(skeleton_doc_1_thinned)
nx_graph_1,pos_1=generate_nx_graph(graph_tab_1_reconnected)

In [41]:
connected_small=list(nx.connected_components(nx_graph_1))

In [51]:
len_connected = [len(connected) for connected in connected_small]

In [48]:
nx_graph_clean=prune_graph(nx_graph_1)

In [57]:
from extract_graph import generate_skeleton
skel_clean=generate_skeleton(nx_graph_clean,dim=skeleton_doc_1_thinned.shape)

In [58]:
from extract_graph import from_sparse_to_graph
panda_graph = from_sparse_to_graph(skel_clean)

In [56]:
skeleton_doc_1_thinned.shape

(20687, 45556)

In [54]:
pos_1[2854]

array([12603, 11744])

In [52]:
len_connected

[6673, 4, 4, 4, 4, 118, 6, 8, 4, 10, 4, 4, 6]

In [46]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.hist(len_connected)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [33]:
plot_nodes_from_list(list(connected_small[11]),pos_1,skeleton_doc_1_thinned.todense())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [38]:
connected_small=list(nx.connected_components(nx_graph_small.todense()))

In [39]:
len(connected_small)

642

In [40]:
skeleton_doc_small

<5000x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 66952 stored elements in Dictionary Of Keys format>

In [43]:
list(connected_small[1])

[160, 123]

In [35]:
pos_small[1306]

array([2240,  413])

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.imshow(skeleton_doc_small.todense())
plt.show()

In [11]:
graph_tab=from_sparse_to_graph(skeleton_doc)

In [ ]:
import cProfile
cProfile.run('from_sparse_to_graph(skeleton_doc)')

In [23]:
nx_graph,pos=generate_nx_graph(graph_tab)

In [13]:
nx_graph=prune_graph(nx_graph)

In [24]:
connected=list(nx.connected_components(nx_graph))

In [25]:
len(connected)

14100

In [ ]:
for edge in nx_graph.subgraph(connected[329]).copy().edges.data('pixel_list'):
    print(edge)

In [ ]:
def zsAlgoIterationOne(image):
    Image_Thinned=sparse.dok_matrix((image.shape[0]+2,image.shape[1]+2), dtype=bool)
    Image_Thinned[1:image.shape[0]+1,1:image.shape[1]+1]=image.copy()
    changing1 = changing2 = 1
    i = 0
    while changing1 or changing2:
        changes_occured = 0
        changing1 = []
        for pos in Image_Thinned.keys():
            x=pos[0]
            y=pos[1]
            P2,P3,P4,P5,P6,P7,P8,P9 = n = neighbours(x, y, Image_Thinned)
            if (Image_Thinned[x,y] == 1 and 2 <= sum(n) <= 6 and transitions(n) == 1 and P2 * P4 * P6 == 0  and P4 * P6 * P8 == 0):
                changing1.append((x,y))
        for x, y in changing1: 
            Image_Thinned[x,y] = 0
            changes_occured = changes_occured + 1
        changing2 = []
        for pos in Image_Thinned.keys():
            x=pos[0]
            y=pos[1]
            P2,P3,P4,P5,P6,P7,P8,P9 = n = neighbours(x, y, Image_Thinned)
            if (Image_Thinned[x,y] == 1 and 2 <= sum(n) <= 6 and transitions(n) == 1 and P2 * P4 * P8 == 0  and P2 * P6 * P8 == 0):
                changing2.append((x,y))    
        for x, y in changing2: 
            Image_Thinned[x,y] = 0
            changes_occured = changes_occured + 1
        i = i + 1
        print("Iteration: ", i , "changes_occured: ", changes_occured)
    return Image_Thinned

def neighbours(x,y,image):
    img = image
    x_1, y_1, x1, y1 = x-1, y-1, x+1, y+1
    return [ img[x_1,y], img[x_1,y1], img[x,y1], img[x1,y1], img[x1,y], img[x,y_1], img[x,y_1], img[x_1,y_1] ]

def transitions(neighbours):
    n = neighbours + neighbours[0:1]
    return sum( (n1, n2) == (0, 1) for n1, n2 in zip(n, n[1:]) )

def foregroundPixels(image):
    fgp = 0
    row, col = image.shape
    for pos in image.keys():
        i=pos[0]
        j=pos[1]
        if(image[i][j]==1):
            fgp = fgp + 1
    return fgp

from copy import copy

In [146]:
time=cProfile.run('generate_nx_graph(graph_tab)')

         14812846 function calls (14672662 primitive calls) in 235.492 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   140184    0.097    0.000    0.274    0.000 <frozen importlib._bootstrap>:1009(_handle_fromlist)
        1    0.025    0.025  235.492  235.492 <string>:1(<module>)
        2    0.000    0.000    0.000    0.000 _asarray.py:14(asarray)
    93456    0.042    0.000    0.042    0.000 _dtype.py:24(_kind_name)
    93456    0.039    0.000    0.039    0.000 _dtype.py:307(_name_includes_bit_suffix)
    93456    0.231    0.000    0.332    0.000 _dtype.py:321(_name_get)
   140184    0.054    0.000    0.192    0.000 abc.py:137(__instancecheck__)
        2    0.000    0.000    0.000    0.000 base.py:1032(__iter__)
   186912    0.184    0.000    0.184    0.000 base.py:1751(is_floating)
   186912    0.127    0.000    0.561    0.000 base.py:256(is_dtype)
   186912    0.272    0.000    1.142    0.000 base.py:2847(get_loc)
